In [1]:
import pandas as pd
import os

# Procesamiento de los datos de las encuestas familiares

## 2006-2010

In [2]:
import os
import pandas as pd

# Ruta base donde están las carpetas 2006–2010
base_path = "../../data"
years_06_10 = list(range(2006, 2011))

### --- 1) FICHERO DE HOGAR (2006–2010) ---

In [3]:
colspecs_hogar_06_10 = [
    (0,4),(4,9),(9,11),(11,12),(12,13),(13,14),(14,15),(15,16),(16,17),
    (17,28),(28,30),(30,31),(31,33),(33,35),(35,37),(37,39),(39,41),(41,43),
    (43,45),(45,47),(47,49),(49,51),(51,53),(53,56),(56,59),(59,61),(61,63),
    (63,65),(65,67),(67,69),(69,71),(71,73),(73,75),(75,77),(77,79),(79,81),
    (81,83),(83,85),(85,87),(87,89),(89,91),(91,93),(93,95),(95,97),(97,99),
    (99,101),(101,103),(103,105),(105,107),(107,109),(109,111),(111,113),(113,115),
    (115,117),(117,119),(119,121),(121,123),(123,125),(125,130),(130,131),(131,133),
    (133,134),(134,136),(136,138),(138,140),(140,142),(142,144),(144,146),(146,148),
    (148,150),(150,152),(152,154),(154,155),(155,157),(157,159),(159,161),(161,163),
    (163,165),(165,168),(168,170),(170,172),(172,174),(174,176),(176,177),(177,178),
    (178,180),(180,182),(182,184),(184,186),(186,188),(188,190),(190,192),(192,194),
    (194,196),(196,198),(198,200),(200,202),(202,204),(204,206),(206,208),(208,210),
    (210,212),(212,214),(214,216),(216,218),(218,220),(220,222),(222,224),(224,226),
    (226,228),(228,230),(230,232),(232,234),(234,236),(236,238),(238,240),(240,242),
    (242,244),(244,246),(246,248),(248,250),(250,252),(252,254),(254,256),(256,258),
    (258,260),(260,262),(262,264),(264,266),(266,268),(268,270),(270,272),(272,274),
    (274,276),(276,278),(278,280),(280,282),(282,284),(284,286),(286,288),(288,290),
    (290,292),(292,294),(294,296),(296,298),(298,300),(300,302),(302,304),(304,320),
    (320,325),(325,341),(341,354),(354,367),(367,380),(380,393),(393,395),(395,397),
    (397,399),(399,401),(401,403),(403,405),(405,407),(407,409),(409,411),(411,416),
    (416,417),(417,419),(419,420),(420,422),(422,425),(425,428),(428,431),(431,434),
    (434,437)
]

column_names_hogar_06_10 = [
    "ANOENC","NUMERO","CCAA","NUTS1","CAPROV","TAMAMU","DENSIDAD","CLAVE","CLATEO",
    "FACTOR","NMIEMB","TAMANO","NMIEMSD","NMIEMHU","NMIEMIN",
    *["NMIEM{}".format(i) for i in range(1,8)], "NHIJOSD","UC1","UC2",
    "PF2TEO","PF2RECO",
    *["TIPHOGAR_{}".format(i) for i in range(1,12)],
    "NUMACTI","NUMOCU","NUMESTU","SITUOCU_HOG","SITUACT_HOG","NORDENSP",
    "EDADSP","SEXOSP","NACIONASP","PAISSP","SITURES_SP","ECIVILSP",
    "NORDENCO_SP","NORDENPA_SP","NORDENMA_SP","ESTUDIOS_SP","ESTUDREDSP",
    "SITUACT_SP","SITURED_SP","PERCEPSP","IMPEXACP_SP","IMPUEXACP_SP",
    "INTERINP_SP","IMPU_INTERPSP","TRABAJO","OCUPA","OCUPARED","ACTESTB",
    "ACTESTB_RED","SITPROF","SECTOR","TIPOCONT","SITSOCI","SITSOCI_RE",
    "REGTEN","TIPOEDIF","ZONARES","TIPOCASA","NHABIT","ANNOCON","SUPERF",
    "AGUACALI","FUENAGUA","CALEF","FUENCALE","DISPOSIOV","NUMOVD",
    *["REGTENV{}".format(i) for i in range(1,10)],
    *["MESESV{}".format(i) for i in range(1,10)],
    *["DIASV{}".format(i) for i in range(1,10)],
    *["AGUACV{}".format(i) for i in range(1,10)],
    *["FUENACV{}".format(i) for i in range(1,10)],
    *["CALEFV{}".format(i) for i in range(1,10)],
    *["FUENCAV{}".format(i) for i in range(1,10)],
    "GASTOT","IMPUTGAS","GASTMON","GASTNOM1","GASTNOM2","GASTNOM3","GASTNOM4",
    "CAPROP","CAJENA","PENSIO","DESEM","OTRSUB","RENTAS","OTROIN","FUENPRIN",
    "FUENPRINRED","IMPEXAC","IMPUEXAC","INTERIN","IMPU_INTER","NUMPERI",
    "COMIMH","COMISD","COMIHU","COMIINV","COMITOT"
]

hogar_06_10 = []
for year in years_06_10:
    path = os.path.join(base_path, str(year), f"Fichero de usuario de hogar a{year}")
    if os.path.exists(path):
        df = pd.read_fwf(path, colspecs=colspecs_hogar_06_10, names=column_names_hogar_06_10, encoding="utf-8")
        df["Año"] = year
        hogar_06_10.append(df)
df_hogar_06_10 = pd.concat(hogar_06_10, ignore_index=True) if hogar_06_10 else None


Corregir parte decimal columna factor

In [4]:
df_hogar_06_10['FACTOR']

0         902614262
1         256638442
2         386234221
3         460834916
4         835798459
            ...    
107598    231101028
107599    977656451
107600    246937427
107601    408839645
107602    523943601
Name: FACTOR, Length: 107603, dtype: int64

### --- 2) FICHERO DE GASTOS (2006–2010) ---

In [5]:
colspecs_gastos_06_10 = [
    (0,4),(4,9),(9,14),(14,29),(29,34),(34,46),(46,61),
    (61,74),(74,87),(87,100),(100,113),(113,126),(126,137)
]
column_names_gastos_06_10 = [
    "ANOENC","NUMERO","CODIGO","GASTO","PORCEN","CANTIDAD","GASTOMON",
    "GASTNOM1","GASTNOM2","GASTNOM3","GASTNOM4","GASTNOM5","FACTOR"
]

gastos_06_10 = []
for year in years_06_10:
    path = os.path.join(base_path, str(year), f"Fichero de usuario de gastos a{year}")
    if os.path.exists(path):
        df = pd.read_fwf(path, colspecs=colspecs_gastos_06_10, names=column_names_gastos_06_10, encoding="utf-8")
        df["Año"] = year
        gastos_06_10.append(df)
df_gastos_06_10 = pd.concat(gastos_06_10, ignore_index=True) if gastos_06_10 else None

In [6]:
df_gastos_06_10

,ANOENC,NUMERO,CODIGO,GASTO,PORCEN,CANTIDAD,GASTOMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,FACTOR,Año
0,2006,1,1112,29415522,0,10354264.0,29415522.0,NaN,NaN,NaN,NaN,NaN,902614262,2006
1,2006,1,1114,2494436,0,705973.0,2494436.0,NaN,NaN,NaN,NaN,NaN,902614262,2006
2,2006,1,1115,57113177,0,NaN,57113177.0,NaN,NaN,NaN,NaN,NaN,902614262,2006
3,2006,1,1117,6000766,0,NaN,6000766.0,NaN,NaN,NaN,NaN,NaN,902614262,2006
4,2006,1,1121,27085813,0,1882593.0,27085813.0,NaN,NaN,NaN,NaN,NaN,902614262,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10349970,2010,22203,11113,136531276,0,NaN,136531276.0,NaN,NaN,NaN,NaN,NaN,523943601,2010
10349971,2010,22203,11115,42028660,0,NaN,42028660.0,NaN,NaN,NaN,NaN,NaN,523943601,2010
10349972,2010,22203,12111,28685991,0,NaN,28685991.0,NaN,NaN,NaN,NaN,NaN,523943601,2010
10349973,2010,22203,12421,20441660,0,NaN,20441660.0,NaN,NaN,NaN,NaN,NaN,523943601,2010


### --- 3) FICHERO DE MIEMBROS (2006–2010) ---

In [7]:
colspecs_miembros_06_10 = [
    (0,4),(4,9),(9,11),(11,13),(13,14),(14,16),(16,18),(18,20),(20,22),(22,24),
    (24,26),(26,28),(28,30),(30,32),(32,34),(34,36),(36,38),(38,40),(40,42),(42,44),
    (44,46),(46,48),(48,50),(50,52),(52,54),(54,56),(56,58),(58,60)
]
column_names_miembros_06_10 = [
    "ANOENC","NUMERO","NORDEN","CATEGMH","SUSPRIN","RELASP","EDAD","SEXO",
    "NACIONA","PAIS","SITURES","ECIVIL","NORDENCO","NORDENPA","NORDENMA",
    "ESTUDIOS","ESTUDRED","SITUACT","SITURED","PERCEP","IMPEXACP","IMPUEXACP",
    "INTERINP","IMPU_INTERP","NINODEP","HIJODEP","ADULTO","FACTOR"
]

miembros_06_10 = []
for year in years_06_10:
    path = os.path.join(base_path, str(year), f"Fichero de usuario de miembros a{year}")
    if os.path.exists(path):
        df = pd.read_fwf(path, colspecs=colspecs_miembros_06_10, names=column_names_miembros_06_10, encoding="utf-8")
        df["Año"] = year
        miembros_06_10.append(df)
df_miembros_06_10 = pd.concat(miembros_06_10, ignore_index=True) if miembros_06_10 else None


In [8]:
df_miembros_06_10

,ANOENC,NUMERO,NORDEN,CATEGMH,SUSPRIN,RELASP,EDAD,SEXO,NACIONA,PAIS,...,PERCEP,IMPEXACP,IMPUEXACP,INTERINP,IMPU_INTERP,NINODEP,HIJODEP,ADULTO,FACTOR,Año
0,2006,1,1,1,1,1,55,6,1,NaN,...,1,NaN,-,9.0,7.0,NaN,6,6,10,2006
1,2006,1,2,1,6,2,50,1,1,NaN,...,1,NaN,-,9.0,3.0,NaN,6,6,10,2006
2,2006,2,1,1,1,1,49,1,1,NaN,...,1,NaN,-,9.0,3.0,NaN,6,6,10,2006
3,2006,2,2,1,6,2,45,6,1,NaN,...,1,NaN,-,9.0,3.0,NaN,6,6,10,2006
4,2006,2,3,1,6,3,22,6,1,NaN,...,6,NaN,NaN,NaN,NaN,NaN,1,1,60,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306024,2010,22202,2,1,6,2,48,6,1,NaN,...,6,NaN,NaN,NaN,NaN,NaN,6,6,10,2010
306025,2010,22202,3,1,6,3,26,6,1,NaN,...,6,NaN,NaN,NaN,NaN,NaN,6,6,10,2010
306026,2010,22202,4,1,6,3,18,6,1,NaN,...,6,NaN,NaN,NaN,NaN,NaN,1,1,60,2010
306027,2010,22203,1,1,1,1,78,1,1,NaN,...,1,NaN,-,9.0,4.0,NaN,6,6,10,2010


## 2011-2015

In [9]:
import os
import pandas as pd

# Ruta base donde están las carpetas 2006, 2007, …, 2015
base_path = "../../data"

years_old = list(range(2011, 2016))

### --- 1) FICHERO DE HOGAR ---


In [10]:
colspecs_hogar = [
    (0, 4), (4, 9), (9,11), (11,12), (12,13), (13,14), (14,15), (15,16), (16,17),
    (17,28), (28,30), (30,31), (31,33), (33,35), (35,37), (37,39), (39,41), (41,43),
    (43,45), (45,47), (47,49), (49,51), (51,53), (53,55), (55,57), (57,59), (59,61),
    (61,63), (63,65), (65,67), (67,69), (69,71), (71,73), (73,75), (75,77), (77,79),
    (79,82), (82,85), (85,87), (87,89), (89,91), (91,93), (93,95), (95,97), (97,99),
    (99,101),(101,103),(103,105),(105,107),(107,109),(109,111),(111,113),(113,115),
    (115,117),(117,119),(119,121),(121,123),(123,125),(125,127),(127,129),(129,131),
    (131,133),(133,135),(135,137),(137,139),(139,141),(141,143),(143,145),(145,147),
    (147,149),(149,151),(151,153),(153,155),(155,157),(157,159),(159,164),(164,165),
    (165,167),(167,168),(168,170),(170,172),(172,174),(174,176),(176,178),(178,180),
    (180,182),(182,184),(184,186),(186,188),(188,189),(189,191),(191,193),(193,195),
    (195,197),(197,199),(199,202),(202,204),(204,206),(206,208),(208,210),(210,211),
    (211,212),(212,214),(214,216),(216,218),(218,220),(220,222),(222,224),(224,226),
    (226,228),(228,230),(230,232),(232,234),(234,236),(236,238),(238,240),(240,242),
    (242,244),(244,246),(246,248),(248,250),(250,252),(252,254),(254,256),(256,258),
    (258,260),(260,262),(262,264),(264,266),(266,268),(268,270),(270,272),(272,274),
    (274,276),(276,278),(278,280),(280,282),(282,284),(284,286),(286,288),(288,290),
    (290,292),(292,294),(294,296),(296,298),(298,300),(300,302),(302,304),(304,306),
    (306,308),(308,310),(310,312),(312,314),(314,316),(316,318),(318,320),(320,322),
    (322,324),(324,326),(326,328),(328,330),(330,332),(332,334),(334,336),(336,338),
    (338,354),(354,359),(359,375),(375,388),(388,401),(401,414),(414,427),(427,429),
    (429,431),(431,433),(433,435),(435,437),(437,439),(439,441),(441,443),(443,445),
    (445,450),(450,451),(451,453),(453,454),(454,456),(456,459),(459,462),(462,465),
    (465,468),(468,471)
]

column_names_hogar = [
    "ANOENC","NUMERO","CCAA","NUTS1","CAPROV","TAMAMU","DENSIDAD","CLAVE","CLATEO",
    "FACTOR","NMIEMB","TAMANO","NMIEMSD","NMIEMHU","NMIEMIN",
    *["NMIEM{}".format(i) for i in range(1,14)],
    "NUMACTI","NUMINACTI","NUMOCU","NUMNOCUP","NUMESTU","NUMNOESTU",
    "NNINOSD","NHIJOSD","UC1","UC2","PF2TEO","PF2RECO",
    *["TIPHOGAR{}".format(i) for i in range(1,12)],
    "SITUOCUHOG","SITUACTHOG","NORDENSP","EDADSP","SEXOSP",
    "PAISNACODSP","NACIONASP","PAISCODSP","SITURESSP","ECIVILLEGALSP",
    "NORDENCOSP","UNIONSP","CONVIVENCIASP","NORDENPASP","PAISPADRESP",
    "NORDENMASP","PAISMADRESP","ESTUDIOSSP","ESTUDREDSP","SITUACTSP","SITUREDSP",
    "OCUSP","JORNADASP","PERCEPSP","IMPEXACPSP","IMPUEXACPSP","INTERINPSP",
    "IMPUINTERPSP","TRABAJO","OCUPA","OCUPARED","ACTESTB","ACTESTBRED",
    "SITPROF","SECTOR","TIPOCONT","SITSOCI","SITSOCIRE","REGTEN","TIPOEDIF",
    "ZONARES","TIPOCASA","NHABIT","ANNOCON","SUPERF","AGUACALI","FUENAGUA",
    "CALEF","FUENCALE","DISPOSIOV","NUMOVD",
    *["REGTENV{}".format(i) for i in range(1,10)],
    *["MESESV{}".format(i) for i in range(1,10)],
    *["DIASV{}".format(i) for i in range(1,10)],
    *["AGUACV{}".format(i) for i in range(1,10)],
    *["FUENACV{}".format(i) for i in range(1,10)],
    *["CALEFV{}".format(i) for i in range(1,10)],
    *["FUENCAV{}".format(i) for i in range(1,10)],
    "GASTOT","IMPUTGAS","GASTMON","GASTNOM1","GASTNOM2","GASTNOM3","GASTNOM4",
    "CAPROP","CAJENA","PENSIO","DESEM","OTRSUB","RENTAS","OTROIN","FUENPRIN",
    "FUENPRINRED","IMPEXAC","IMPUEXAC","INTERIN","IMPUINTER","NUMPERI",
    "COMIMH","COMISD","COMIHU","COMIINV","COMITOT"
]

hogar_old_dfs = []
for año in years_old:
    path = os.path.join(base_path, str(año), f"Fichero de usuario de hogar a{año}")
    if os.path.exists(path):
        df = pd.read_fwf(path, colspecs=colspecs_hogar, names=column_names_hogar, encoding="utf-8")
        df["Año"] = año
        hogar_old_dfs.append(df)
df_hogar_old = pd.concat(hogar_old_dfs, ignore_index=True) if hogar_old_dfs else None

Corregir decimales implicitos columna factor

In [11]:
df_hogar_old

,ANOENC,NUMERO,CCAA,NUTS1,CAPROV,TAMAMU,DENSIDAD,CLAVE,CLATEO,FACTOR,...,IMPUEXAC,INTERIN,IMPUINTER,NUMPERI,COMIMH,COMISD,COMIHU,COMIINV,COMITOT,Año
0,2011,1,10,5,6,2,2,1,1,745065408,...,1,7,6,2,48,0,0,0,48,2011
1,2011,2,8,4,6,3,3,1,1,713066493,...,1,3,6,1,83,0,0,0,83,2011
2,2011,3,13,3,6,4,3,1,1,928101740,...,6,7,6,2,65,0,0,0,65,2011
3,2011,4,9,5,6,5,3,2,2,2076523457,...,6,3,6,1,140,0,0,12,152,2011
4,2011,5,17,2,1,1,1,1,1,138126309,...,1,6,6,2,84,0,0,0,84,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109816,2015,22126,5,7,6,2,2,1,1,4379166284,...,1,3,6,3,0,0,0,0,0,2015
109817,2015,22127,9,5,6,1,1,1,2,2251485110,...,1,3,6,1,56,0,0,0,56,2015
109818,2015,22128,9,5,6,3,2,1,1,1617862740,...,6,4,6,2,54,0,0,16,70,2015
109819,2015,22129,12,1,6,5,3,2,2,1233295394,...,1,1,6,1,14,0,0,0,14,2015


### --- 2) FICHERO DE GASTOS ---


In [12]:

colspecs_gastos = [
    (0,4),(4,9),(9,14),(14,29),(29,34),(34,46),(46,61),
    (61,74),(74,87),(87,100),(100,113),(113,126),(126,137)
]
column_names_gastos = [
    "ANOENC","NUMERO","CODIGO","GASTO","PORCEN","CANTIDAD","GASTOMON",
    "GASTNOM1","GASTNOM2","GASTNOM3","GASTNOM4","GASTNOM5","FACTOR"
]

gastos_old_dfs = []
for año in years_old:
    path = os.path.join(base_path, str(año), f"Fichero de usuario de gastos a{año}")
    if os.path.exists(path):
        df = pd.read_fwf(path, colspecs=colspecs_gastos, names=column_names_gastos, encoding="utf-8")
        df["Año"] = año
        gastos_old_dfs.append(df)
df_gastos_old = pd.concat(gastos_old_dfs, ignore_index=True) if gastos_old_dfs else None



In [13]:
df_gastos_old

,ANOENC,NUMERO,CODIGO,GASTO,PORCEN,CANTIDAD,GASTOMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,FACTOR,Año
0,2011,1,1112,17975218,0,6410216.0,17975218.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
1,2011,1,1113,23693520,0,4661976.0,23693520.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
2,2011,1,1115,50528045,0,NaN,50528045.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
3,2011,1,1117,1942490,0,NaN,1942490.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
4,2011,1,1125,33205115,0,3884980.0,33205115.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10160616,2015,22130,12421,7459365,0,NaN,7459365.0,NaN,NaN,NaN,NaN,NaN,559172763,2015
10160617,2015,22130,12441,18323532,0,NaN,18323532.0,NaN,NaN,NaN,NaN,NaN,559172763,2015
10160618,2015,22130,12461,22559266,0,NaN,22559266.0,NaN,NaN,NaN,NaN,NaN,559172763,2015
10160619,2015,22130,12611,33608236,0,NaN,33608236.0,NaN,NaN,NaN,NaN,NaN,559172763,2015


### --- 3) FICHERO DE MIEMBROS ---


In [14]:
colspecs_miembros = [
    (0,4),(4,9),(9,11),(11,13),(13,14),(14,16),(16,18),(18,20),(20,22),(22,24),
    (24,26),(26,28),(28,30),(30,32),(32,34),(34,36),(36,38),(38,40),(40,42),(42,44),
    (44,46),(46,48),(48,50),(50,52),(52,54),(54,56),(56,58),(58,63),(63,64),(64,66),
    (66,67),(67,69),(69,71),(71,73),(73,84)
]
column_names_miembros = [
    "ANOENC","NUMERO","NORDEN","CATEGMH","SUSPRIN","RELASP","EDAD","SEXO",
    "PAISNACIM","NACIONA","PAISNACION","SITURES","ECIVILLEGAL","NORDENCO",
    "UNION","CONVIVENCIA","NORDENPA","PAISPADRE","NORDENMA","PAISMADRE",
    "ESTUDIOS","ESTUDRED","SITUACT","SITURED","OCU","JORNADA","PERCEP",
    "IMPEXACP","IMPUEXACP","INTERINP","IMPUINTERP","NINODEP","HIJODEP",
    "ADULTO","FACTOR"
]

miembros_old_dfs = []
for año in years_old:
    path = os.path.join(base_path, str(año), f"Fichero de usuario de miembros a{año}")
    if os.path.exists(path):
        df = pd.read_fwf(path, colspecs=colspecs_miembros, names=column_names_miembros, encoding="utf-8")
        df["Año"] = año
        miembros_old_dfs.append(df)
df_miembros_old = pd.concat(miembros_old_dfs, ignore_index=True) if miembros_old_dfs else None


In [15]:
df_miembros_old

,ANOENC,NUMERO,NORDEN,CATEGMH,SUSPRIN,RELASP,EDAD,SEXO,PAISNACIM,NACIONA,...,PERCEP,IMPEXACP,IMPUEXACP,INTERINP,IMPUINTERP,NINODEP,HIJODEP,ADULTO,FACTOR,Año
0,2011,1,1,1,1,1,35,1,1,1,...,1,-9.0,NaN,4.0,NaN,6,6,1,745065408,2011
1,2011,1,2,1,6,2,32,6,1,1,...,1,-9.0,NaN,3.0,NaN,6,6,1,745065408,2011
2,2011,1,3,1,6,3,2,1,1,1,...,6,NaN,NaN,NaN,NaN,1,1,6,745065408,2011
3,2011,1,4,1,6,3,2,6,1,1,...,6,NaN,NaN,NaN,NaN,1,1,6,745065408,2011
4,2011,2,1,1,1,1,45,1,1,1,...,1,-9.0,NaN,3.0,NaN,6,6,1,713066493,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301503,2015,22129,1,1,1,1,50,1,1,1,...,1,-9.0,NaN,1.0,NaN,6,6,1,1233295394,2015
301504,2015,22130,1,1,1,1,52,1,1,1,...,1,1067.0,NaN,3.0,NaN,6,6,1,559172763,2015
301505,2015,22130,2,1,6,2,49,6,1,1,...,1,343.0,NaN,1.0,NaN,6,6,1,559172763,2015
301506,2015,22130,3,1,6,3,18,6,1,1,...,6,NaN,NaN,NaN,NaN,1,1,6,559172763,2015


## 2016-2023

In [16]:
# Definir la ruta base donde están los datos
base_path = "../../data"

# Estructura de los archivos
years = list(range(2016, 2024))  # De 2016 a 2023
file_types = ["EPFgastos", "EPFhogar", "EPFmhogar"]

# Diccionario para almacenar los DataFrames
dataframes = {ftype: [] for ftype in file_types}

# Leer los archivos y almacenarlos
for year in years:
    year_path = os.path.join(base_path, str(year))
    for ftype in file_types:
        file_name = f"{ftype}_{year}.csv"
        file_path = os.path.join(year_path, file_name)
        
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, low_memory=False, encoding="utf-8", sep="\t")  # Cargar el CSV
            df["Año"] = year  # Agregar una columna de año
            dataframes[ftype].append(df)

# Unir los archivos en DataFrames únicos por tipo
df_gastos = pd.concat(dataframes["EPFgastos"], ignore_index=True) if dataframes["EPFgastos"] else None
df_hogar = pd.concat(dataframes["EPFhogar"], ignore_index=True) if dataframes["EPFhogar"] else None
df_mhogar = pd.concat(dataframes["EPFmhogar"], ignore_index=True) if dataframes["EPFmhogar"] else None


In [17]:
df_gastos.head()

,ANOENC,NUMERO,CODIGO,GASTO,PORCENDES,PORCENIMP,CANTIDAD,GASTOMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,FACTOR,Año
0,2016,1,01111,37.22,100.00,100.00,NaN,37.22,NaN,NaN,NaN,NaN,NaN,742.998897,2016
1,2016,1,01112,14.79,100.00,100.00,NaN,14.79,NaN,NaN,NaN,NaN,NaN,742.998897,2016
2,2016,1,01113,113964.81,0.50,83.00,45869.56,113964.81,NaN,NaN,NaN,NaN,NaN,742.998897,2016
3,2016,1,01114,54614.37,0.69,0.69,5811.31,54614.37,NaN,NaN,NaN,NaN,NaN,742.998897,2016
4,2016,1,01115,73.85,100.00,100.00,NaN,73.85,NaN,NaN,NaN,NaN,NaN,742.998897,2016


In [18]:
df_gastos.tail()

,ANOENC,NUMERO,CODIGO,GASTO,PORCENDES,PORCENIMP,CANTIDAD,GASTOMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,FACTOR,Año
14145054,2023,20707,12521,106228.09,0.0,0.0,NaN,106228.09,NaN,NaN,NaN,NaN,NaN,544.759442,2023
14145055,2023,20707,12522,136189.86,0.0,0.0,NaN,136189.86,NaN,NaN,NaN,NaN,NaN,544.759442,2023
14145056,2023,20707,12541,163427.83,0.0,0.0,NaN,163427.83,NaN,NaN,NaN,NaN,NaN,544.759442,2023
14145057,2023,20707,12620,47712.61,0.0,0.0,NaN,47712.61,NaN,NaN,NaN,NaN,NaN,544.759442,2023
14145058,2023,20707,12702,834970.72,0.0,0.0,NaN,834970.72,NaN,NaN,NaN,NaN,NaN,544.759442,2023


In [19]:
df_hogar

,ANOENC,NUMERO,CCAA,NUTS1,CAPROV,TAMAMU,DENSIDAD,CLAVE,CLATEO,FACTOR,...,FUENPRINRED,IMPEXAC,INTERIN,NUMPERI,COMIMH,COMISD,COMIHU,COMIINV,COMITOT,Año
0,2016,1,7,4,6,5,3,2,2,742.998897,...,1,1225,3,2,8,0,0,0,8,2016
1,2016,2,12,1,6,2,1,1,2,1849.021419,...,3,676,2,2,38,0,0,0,38,2016
2,2016,3,13,3,1,1,1,1,1,1580.183563,...,2,5868,8,4,89,0,0,10,99,2016
3,2016,4,8,4,6,5,3,2,2,846.366967,...,2,3769,7,4,116,0,0,0,116,2016
4,2016,5,13,3,6,1,1,2,2,1359.668063,...,2,1271,3,1,84,0,0,0,84,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166117,2023,20703,16,2,1,1,1,1,2,314.700969,...,3.0,2603,6,1,14,0,0,0,14,2023
166118,2023,20704,6,1,6,5,3,2,2,360.352153,...,3.0,2662,6,2,84,0,0,0,84,2023
166119,2023,20705,9,5,6,4,2,1,2,1420.714232,...,3.0,2400,5,2,28,0,0,0,28,2023
166120,2023,20706,7,4,6,5,3,1,1,432.570078,...,1.0,3067,7,2,52,0,0,4,56,2023


# Procesamiento del Fichero de Hogar

In [20]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
df_hogar_06_10['NUMNOESTU'] = df_hogar_06_10['NMIEMB'] - df_hogar_06_10['NUMESTU']

In [21]:
df_hogar.columns

Index(['ANOENC', 'NUMERO', 'CCAA', 'NUTS1', 'CAPROV', 'TAMAMU', 'DENSIDAD',
       'CLAVE', 'CLATEO', 'FACTOR',
       ...
       'FUENPRINRED', 'IMPEXAC', 'INTERIN', 'NUMPERI', 'COMIMH', 'COMISD',
       'COMIHU', 'COMIINV', 'COMITOT', 'Año'],
      dtype='object', length=189)

In [22]:
# Agregar en df_hogar_06_10, cambiar nombre de NMIEM7 a la columna 'NINOSD'
df_hogar_06_10.rename(columns={"NMIEM7": "NNINOSD"}, inplace=True)

In [23]:
# Creamos la columna 'GASTNOM4' como la suma de 'GASTNOM1', 'GASTNOM2' y 'GASTNOM3', 'GASTNOM4 Y 'GASTNOM5'
df_hogar_06_10[["GASTNOM1", "GASTNOM2", "GASTNOM3", "GASTNOM4"]] = df_hogar_06_10[["GASTNOM1", "GASTNOM2", "GASTNOM3", "GASTNOM4"]].fillna(0)
df_hogar_old[["GASTNOM1", "GASTNOM2", "GASTNOM3", "GASTNOM4"]] = df_hogar_old[["GASTNOM1", "GASTNOM2", "GASTNOM3", "GASTNOM4"]].fillna(0)
df_hogar[["GASTNOM1", "GASTNOM2", "GASTNOM3", "GASTNOM4"]] = df_hogar[["GASTNOM1", "GASTNOM2", "GASTNOM3", "GASTNOM4"]].fillna(0)

df_hogar_06_10["GASTNOMON"] = df_hogar_06_10[["GASTNOM1", "GASTNOM2", "GASTNOM3", "GASTNOM4"]].sum(axis=1)
df_hogar_old["GASTNOMON"] = df_hogar_old[["GASTNOM1", "GASTNOM2", "GASTNOM3", "GASTNOM4"]].sum(axis=1)
df_hogar["GASTNOMON"] = df_hogar[["GASTNOM1", "GASTNOM2", "GASTNOM3", "GASTNOM4"]].sum(axis=1)

Seleccionamos las columnas que almacenaremos en nuestro datalake

In [24]:
columnas_df_hogar = [
    # 1️⃣ Información General
    "ANOENC", "NUMERO", "CCAA", "CAPROV", "TAMAMU", "DENSIDAD",

    # 2️⃣ Características del Hogar (excluyendo 'TAMAÑO')
    "NUMACTI", "NUMOCU", "NNINOSD", "NHIJOSD", "NUMESTU", "NUMNOESTU",

    # 3️⃣ Datos del Sustentador Principal
    "EDADSP", "NACIONASP", "ESTUDREDSP",

    # 4️⃣ Características de la Vivienda
    "REGTEN", "ZONARES", "SUPERF", "AGUACALI", "CALEF", "TIPOCASA",

    # 5️⃣ Otras Viviendas a Disposición del Hogar
    "DISPOSIOV", "NUMOVD",

    # 7️⃣ Ingresos Regulares del Hogar
    "IMPEXAC", "FUENPRINRED",

    # Gasto por hogar
    "GASTOT", "GASTMON", "GASTNOMON", "FACTOR", "NUMPERI",

    # Habitos de Consumo
    "COMITOT"
]

df_hogar_filtrado = df_hogar[columnas_df_hogar]
df_hogar_filtrado = df_hogar_filtrado.copy()

df_hogar_old_filtrado = df_hogar_old[columnas_df_hogar]
df_hogar_old_filtrado = df_hogar_old_filtrado.copy()

df_hogar_06_10_filtrado = df_hogar_06_10[columnas_df_hogar]
df_hogar_06_10_filtrado = df_hogar_06_10_filtrado.copy()



In [25]:
df_miembros_old.isna().sum()
# Value counts de lac olumna PERCEP
df_miembros_old['PERCEP'].value_counts()

 1    178599
 6    116727
-9      6182
Name: PERCEP, dtype: int64

In [26]:
df_miembros_06_10.isna().sum()
df_miembros_06_10['PERCEP'].value_counts()

 1    183151
 6    114117
-9      8761
Name: PERCEP, dtype: int64

In [27]:
df_mhogar.isna().sum()
df_mhogar['PERCEP'].value_counts()

 1    273542
 6    148267
-9      7356
Name: PERCEP, dtype: int64

In [28]:
import numpy as np

# Convertir los -9 de las columnas PERCEP, SEXO, ADULTO, EDAD a NaN
def convert_neg9_to_nan(df, columns):
    for col in columns:
        df[col] = df[col].replace(-9, np.nan)
    return df

# Convertir los -9 a NaN en las columnas especificadas
columns_to_convert = ["PERCEP", "SEXO", "ADULTO", "EDAD"]
df_miembros_old = convert_neg9_to_nan(df_miembros_old, columns_to_convert)
df_miembros_06_10 = convert_neg9_to_nan(df_miembros_06_10, columns_to_convert)
df_mhogar = convert_neg9_to_nan(df_mhogar, columns_to_convert)

# Eliminar los -9 de la columna TIPOCASA
def remove_neg9_from_tipocasa(df):
    df['TIPOCASA'] = df['TIPOCASA'].replace(-9, np.nan)
    return df

df_hogar_old = remove_neg9_from_tipocasa(df_hogar_old)
df_hogar_06_10 = remove_neg9_from_tipocasa(df_hogar_06_10)
df_hogar = remove_neg9_from_tipocasa(df_hogar)



Identificamos desde el fichero de miembros los adultos y sexos de los miembros

In [29]:
import pandas as pd

def merge_hogar_miembros(
    hogar_df, miembros_df,
    hogar_key="NUMERO", hogar_year="ANOENC",
    miembro_key="NUMERO", miembro_year="Año",
    umbral_anciano=65
):
    """
    Para cada hogar en 'hogar_df', añade columnas con los recuentos por categoría
    de miembros (adultos, niños y ancianos, hombres y mujeres) basándose en 'miembros_df'.
    Además, cuenta las categorías de percepción (PERCEP) por sexo.
    Los miembros con 'edad' > umbral_anciano se consideran ancianos.
    Las categorías de percepción se definen como:
      - PERCEP == 1 y SEXO == 1 -> PERCEP_MAS
      - PERCEP == 6 y SEXO == 1 -> NO_PERCEP_MAS
      - PERCEP == 1 y SEXO == 6 -> PERCEP_FEM
      - PERCEP == 6 y SEXO == 6 -> NO_PERCEP_FEM
    """

    # 1) Copia y convierte ADULTO, SEXO, EDAD y PERCEP a int (descarta NaN donde no sea válido)
    dfm = miembros_df.copy()
    dfm['ADULTO'] = pd.to_numeric(dfm['ADULTO'], errors='coerce').astype('Int64')
    dfm['SEXO']   = pd.to_numeric(dfm['SEXO'],   errors='coerce').astype('Int64')
    dfm['EDAD']   = pd.to_numeric(dfm['EDAD'],   errors='coerce').astype('Int64')
    dfm['PERCEP'] = pd.to_numeric(dfm['PERCEP'], errors='coerce').astype('Int64')

    # 2) Filtrar sólo valores válidos en ADULTO, SEXO y PERCEP
    dfm = dfm[
        dfm['ADULTO'].isin([1,6]) &
        dfm['SEXO'].isin([1,6]) &
        dfm['PERCEP'].isin([1,6])
    ].copy()

    # 3) Clasificar por edad/adulto y sexo
    def clasificar(row):
        if pd.notna(row['EDAD']) and row['EDAD'] > umbral_anciano:
            return 'ANC_MAS' if row['SEXO'] == 1 else 'ANC_FEM'
        if   row['ADULTO'] == 1 and row['SEXO'] == 1: return 'NADUL_MAS'
        elif row['ADULTO'] == 1 and row['SEXO'] == 6: return 'NADUL_FEM'
        elif row['ADULTO'] == 6 and row['SEXO'] == 1: return 'NNINO_MAS'
        elif row['ADULTO'] == 6 and row['SEXO'] == 6: return 'NNINO_FEM'
        else: return None

    dfm['categoria'] = dfm.apply(clasificar, axis=1)

    # 4) Contar por hogar y año para categorías de edad
    conteo = (
        dfm
        .groupby([miembro_key, miembro_year, 'categoria'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )

    # 5) Asegurar columnas siempre presentes (edad)
    expected_age = ['NADUL_MAS','NADUL_FEM','NNINO_MAS','NNINO_FEM','ANC_MAS','ANC_FEM']
    for col in expected_age:
        if col not in conteo:
            conteo[col] = 0

    # 6) Renombrar la columna de año para casar con hogar_df
    conteo = conteo.rename(columns={miembro_year: hogar_year})

    # 7) Fusionar conteo de edad con hogar_df
    out = hogar_df.merge(
        conteo[[miembro_key, hogar_year] + expected_age],
        on=[hogar_key, hogar_year],
        how='left'
    )
    # Rellenar nulos y convertir a int
    for col in expected_age:
        out[col] = out[col].fillna(0).astype(int)

    # 8) Clasificar por percepción y sexo
    def clasificar_percep(row):
        if row['PERCEP'] == 1 and row['SEXO'] == 1:
            return 'PERCEP_MAS'
        elif row['PERCEP'] == 6 and row['SEXO'] == 1:
            return 'NO_PERCEP_MAS'
        elif row['PERCEP'] == 1 and row['SEXO'] == 6:
            return 'PERCEP_FEM'
        elif row['PERCEP'] == 6 and row['SEXO'] == 6:
            return 'NO_PERCEP_FEM'
        else:
            return None

    dfm['percep_categoria'] = dfm.apply(clasificar_percep, axis=1)

    # 9) Contar por hogar y año para percepción
    perc_count = (
        dfm
        .groupby([miembro_key, miembro_year, 'percep_categoria'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )

    # 10) Asegurar columnas siempre presentes (percepción)
    expected_percep = ['PERCEP_MAS','NO_PERCEP_MAS','PERCEP_FEM','NO_PERCEP_FEM']
    for col in expected_percep:
        if col not in perc_count:
            perc_count[col] = 0

    # 11) Renombrar año para casar con hogar_df
    perc_count = perc_count.rename(columns={miembro_year: hogar_year})

    # 12) Fusionar conteo de percepción con el resultado previo
    out = out.merge(
        perc_count[[miembro_key, hogar_year] + expected_percep],
        on=[hogar_key, hogar_year],
        how='left'
    )
    for col in expected_percep:
        out[col] = out[col].fillna(0).astype(int)

    return out

In [30]:
df_hogar_old_filtrado   = merge_hogar_miembros(df_hogar_old_filtrado, df_miembros_old)
df_hogar_06_10_filtrado = merge_hogar_miembros(df_hogar_06_10_filtrado, df_miembros_06_10)
df_hogar_filtrado       = merge_hogar_miembros(df_hogar_filtrado, df_mhogar)

In [31]:
pd.set_option('display.max_columns', None)  
df_hogar_old_filtrado.head()

,ANOENC,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NNINOSD,NHIJOSD,NUMESTU,NUMNOESTU,EDADSP,NACIONASP,ESTUDREDSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOMON,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM
0,2011,1,10,6,2,2,2,2,2,2,0,2,35,1,4,1,2,200,1,1,1,1,1.0,3236,2.0,3642224681,2.874596e+09,7.676286e+08,745065408,2,48,1,1,1,1,0,0,1,1,1,1
1,2011,2,8,6,3,3,2,1,0,0,0,3,45,1,2,2,3,150,1,1,2,6,NaN,1224,1.0,2897830727,2.581191e+09,3.166398e+08,713066493,1,83,1,2,0,0,0,0,1,0,0,2
2,2011,3,13,6,4,3,2,1,1,1,0,2,43,1,4,2,3,200,1,1,1,6,NaN,3700,2.0,3963687846,2.881534e+09,1.082154e+09,928101740,2,65,1,1,1,0,0,0,1,1,1,0
3,2011,4,9,6,5,3,1,1,3,2,1,2,32,2,4,3,3,65,1,6,2,6,NaN,1200,2.0,4923708876,4.923709e+09,0.000000e+00,2076523457,1,152,1,1,1,2,0,0,1,1,0,3
4,2011,5,17,1,1,1,2,2,0,0,0,3,53,1,4,1,3,119,1,1,2,1,1.0,2675,1.0,528585005,3.408333e+08,1.877517e+08,138126309,2,84,2,1,0,0,0,0,2,0,0,1


In [32]:
df_hogar_06_10_filtrado.head()

,ANOENC,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NNINOSD,NHIJOSD,NUMESTU,NUMNOESTU,EDADSP,NACIONASP,ESTUDREDSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOMON,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM
0,2006,1,9,6,2,1,2,2,0,0,0,2,55,1,4,2,3,70,1,1,2,6,NaN,5020,1.0,4260498056,3.669192e+09,591306213.0,902614262,2,58,0,0,1,1,0,0,1,0,1,0
1,2006,2,14,6,4,2,2,2,2,2,2,2,49,1,4,1,7,150,1,1,1,6,NaN,2102,2.0,1096578320,9.753475e+08,121230867.0,256638442,2,112,0,2,1,1,0,0,1,0,1,2
2,2006,3,15,6,5,3,2,2,0,0,0,3,47,1,4,1,5,80,1,1,2,6,NaN,2257,1.0,833188242,6.341741e+08,199014135.0,386234221,2,82,0,0,2,1,0,0,2,0,0,1
3,2006,4,9,1,1,1,1,1,1,1,0,2,49,1,4,3,3,65,1,1,2,6,NaN,1661,2.0,1157571243,1.157571e+09,0.0,460834916,1,36,0,1,0,1,0,0,0,0,1,1
4,2006,5,9,6,5,3,1,1,2,2,0,4,38,1,2,2,7,120,1,1,1,6,NaN,1082,2.0,1935652886,1.532995e+09,402657609.0,835798459,1,72,1,1,1,1,0,0,1,1,0,2


In [33]:
df_hogar_filtrado.head()

,ANOENC,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NNINOSD,NHIJOSD,NUMESTU,NUMNOESTU,EDADSP,NACIONASP,ESTUDREDSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOMON,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM
0,2016,1,7,6,5,3,2,2,0,0,0,2,66,1,1,2,5,96,1,1,2,6,NaN,1225,1,14239915.96,3873629.52,10366286.43,742.998897,2,8,0,1,0,0,1,0,1,0,1,0
1,2016,2,12,6,2,1,2,1,0,0,0,2,35,1,4,1,3,-9,1,1,2,6,NaN,676,3,82364853.85,71862358.20,10502495.65,1849.021419,2,38,1,1,0,0,0,0,1,0,1,0
2,2016,3,13,1,1,1,4,4,0,0,0,4,64,1,2,5,3,155,1,1,2,6,NaN,5868,2,95097473.22,73436857.99,21660615.23,1580.183563,4,99,3,1,0,0,0,0,3,0,1,0
3,2016,4,8,6,5,3,4,2,0,0,0,4,60,1,2,2,7,150,1,1,2,6,NaN,3769,2,25251223.53,20479374.41,4771849.12,846.366967,4,116,2,2,0,0,0,0,2,0,2,0
4,2016,5,13,6,1,1,1,1,1,1,0,3,37,2,3,3,3,98,1,1,2,6,NaN,1271,2,39117817.76,39117817.76,0.00,1359.668063,1,84,1,1,0,1,0,0,1,0,0,2


Unimos todos los años en un solo dataframe

In [34]:
# Unir los DataFrames filtrados concatenar por filas
df_hogar_filtrado = pd.concat([df_hogar_06_10_filtrado, df_hogar_old_filtrado, df_hogar_filtrado], ignore_index=True)

In [35]:
df_hogar_filtrado.head()

,ANOENC,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NNINOSD,NHIJOSD,NUMESTU,NUMNOESTU,EDADSP,NACIONASP,ESTUDREDSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOMON,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM
0,2006,1,9,6,2,1,2,2,0,0,0,2,55,1,4,2,3,70,1,1,2,6,NaN,5020,1.0,4.260498e+09,3.669192e+09,591306213.0,902614262.0,2,58,0,0,1,1,0,0,1,0,1,0
1,2006,2,14,6,4,2,2,2,2,2,2,2,49,1,4,1,7,150,1,1,1,6,NaN,2102,2.0,1.096578e+09,9.753475e+08,121230867.0,256638442.0,2,112,0,2,1,1,0,0,1,0,1,2
2,2006,3,15,6,5,3,2,2,0,0,0,3,47,1,4,1,5,80,1,1,2,6,NaN,2257,1.0,8.331882e+08,6.341741e+08,199014135.0,386234221.0,2,82,0,0,2,1,0,0,2,0,0,1
3,2006,4,9,1,1,1,1,1,1,1,0,2,49,1,4,3,3,65,1,1,2,6,NaN,1661,2.0,1.157571e+09,1.157571e+09,0.0,460834916.0,1,36,0,1,0,1,0,0,0,0,1,1
4,2006,5,9,6,5,3,1,1,2,2,0,4,38,1,2,2,7,120,1,1,1,6,NaN,1082,2.0,1.935653e+09,1.532995e+09,402657609.0,835798459.0,1,72,1,1,1,1,0,0,1,1,0,2


### Limpieza de datos

Convertimos en variables binarias manejables, las variables binarias

In [36]:
df_hogar_filtrado.loc[:, 'CAPROV'] = df_hogar_filtrado['CAPROV'].replace({1: 1, 6: 0})
df_hogar_filtrado.loc[:, 'AGUACALI'] = df_hogar_filtrado['AGUACALI'].replace({1: 1, 6: 0})
df_hogar_filtrado.loc[:, 'CALEF'] = df_hogar_filtrado['CALEF'].replace({1: 1, 6: 0})
df_hogar_filtrado.loc[:, 'DISPOSIOV'] = df_hogar_filtrado['DISPOSIOV'].replace({1: 1, 6: 0})


Manejamos la columna NACIONASP, la convertiremos en dummies:

- NACION_ESP = 1 si la persona tiene nacionalidad española (NACIONASP == 1 o 3).
- NACION_EXT = 1 si la persona tiene nacionalidad extranjera (NACIONASP == 2 o 3).
- Si tiene doble nacionalidad (NACIONASP == 3), ambas serán 1.

In [37]:
df_hogar_filtrado['NACIONASP'].value_counts()

1    355983
2     19919
3      7644
Name: NACIONASP, dtype: int64

In [38]:
df_hogar_filtrado['NACION_ESP'] = df_hogar_filtrado['NACIONASP'].isin([1, 3]).astype(int)
df_hogar_filtrado['NACION_EXT'] = df_hogar_filtrado['NACIONASP'].isin([2, 3]).astype(int)

df_hogar_filtrado.drop(columns=['NACIONASP'], inplace=True)

Estudios del sustentador principal, lo modificaremos a si tiene educación superior o no, y la situación laboral a si está trabajando o no.

In [39]:
df_hogar_filtrado['EDUC_SUPERIOR'] = (df_hogar_filtrado['ESTUDREDSP'] == 4).astype(int)
df_hogar_filtrado.drop(columns=['ESTUDREDSP'], inplace=True)

In [40]:
# La variable TAMAMU esta categorica de 1 a 5 y se le da la vuelta a la escala
df_hogar_filtrado['TAMAMU'] = df_hogar_filtrado['TAMAMU'].replace({1: 5, 2: 4, 3: 3, 4: 2, 5: 1})

# Lo mismo para la variable DENSIDAD
df_hogar_filtrado['DENSIDAD'] = df_hogar_filtrado['DENSIDAD'].replace({1: 3, 2: 2, 3: 1})

# Lo mismo para la variable ZONARES del 1 al 7
df_hogar_filtrado['ZONARES'] = df_hogar_filtrado['ZONARES'].replace({1: 7, 2: 6, 3: 5, 4: 4, 5: 3, 6: 2, 7: 1})

# Lo mismo para la variable REGTEN del 1 al 6
df_hogar_filtrado['REGTEN'] = df_hogar_filtrado['REGTEN'].replace({1: 6, 2: 5, 3: 4, 4: 3, 5: 2, 6: 1})

# Si Disposiov es '0', numovd poner 0
df_hogar_filtrado.loc[df_hogar_filtrado['DISPOSIOV'] == 0, 'NUMOVD'] = 0

# Si NUMPERI es b, poner 0
df_hogar_filtrado['NUMPERI'] = df_hogar_filtrado['NUMPERI'].replace({'b': 0}).astype(int)

# La variable TIPOCASA es categorica de 1 a 3 y se le da la vuelta a la escala
df_hogar_filtrado['TIPOCASA'] = df_hogar_filtrado['TIPOCASA'].replace({1: 3, 2: 2, 3: 1})



In [41]:
df_hogar_filtrado.head()

,ANOENC,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NNINOSD,NHIJOSD,NUMESTU,NUMNOESTU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOMON,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,NACION_EXT,EDUC_SUPERIOR
0,2006,1,9,0,4,3,2,2,0,0,0,2,55,5,5,70,1,1,2,0,0.0,5020,1.0,4.260498e+09,3.669192e+09,591306213.0,902614262.0,2,58,0,0,1,1,0,0,1,0,1,0,1,0,1
1,2006,2,14,0,2,2,2,2,2,2,2,2,49,6,1,150,1,1,3,0,0.0,2102,2.0,1.096578e+09,9.753475e+08,121230867.0,256638442.0,2,112,0,2,1,1,0,0,1,0,1,2,1,0,1
2,2006,3,15,0,1,1,2,2,0,0,0,3,47,6,3,80,1,1,2,0,0.0,2257,1.0,8.331882e+08,6.341741e+08,199014135.0,386234221.0,2,82,0,0,2,1,0,0,2,0,0,1,1,0,1
3,2006,4,9,1,5,3,1,1,1,1,0,2,49,4,5,65,1,1,2,0,0.0,1661,2.0,1.157571e+09,1.157571e+09,0.0,460834916.0,1,36,0,1,0,1,0,0,0,0,1,1,1,0,1
4,2006,5,9,0,1,1,1,1,2,2,0,4,38,5,1,120,1,1,3,0,0.0,1082,2.0,1.935653e+09,1.532995e+09,402657609.0,835798459.0,1,72,1,1,1,1,0,0,1,1,0,2,1,0,0


In [42]:
df_hogar_filtrado['FUENPRINRED'].astype(str).str.lower().value_counts()


2.0     112091
3.0      91493
 2       69217
 3       57799
1.0      30870
 1       16399
-9.0      2441
nan       1236
-9        1076
           924
Name: FUENPRINRED, dtype: int64

In [43]:
# Asegúrate de que la columna sea string
df_hogar_filtrado['FUENPRINRED'] = df_hogar_filtrado['FUENPRINRED'].astype(str).str.strip().str.lower()

# Definimos nombres reducidos
short_map = {
    '1':  'autonomYRenta',      # Trabajo por cuenta propia y rentas
    '2':  'asalariado',     # Trabajo por cuenta ajena
    '3':  'pension',       # pension, subsidios y otras prestaciones
    '1.0':  'autonomYRenta',      # Trabajo por cuenta propia y rentas
    '2.0':  'asalariado',     # Trabajo por cuenta ajena
    '3.0':  'pension', 
}

# Aplicamos el mapeo
df_hogar_filtrado['FUENPRINRED'] = df_hogar_filtrado['FUENPRINRED'].map(short_map)

df_hogar_filtrado['FUENPRINRED'].value_counts()

asalariado       181308
pension          149292
autonomYRenta     47269
Name: FUENPRINRED, dtype: int64

In [44]:
df_hogar_filtrado.head()

,ANOENC,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NNINOSD,NHIJOSD,NUMESTU,NUMNOESTU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOMON,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,NACION_EXT,EDUC_SUPERIOR
0,2006,1,9,0,4,3,2,2,0,0,0,2,55,5,5,70,1,1,2,0,0.0,5020,autonomYRenta,4.260498e+09,3.669192e+09,591306213.0,902614262.0,2,58,0,0,1,1,0,0,1,0,1,0,1,0,1
1,2006,2,14,0,2,2,2,2,2,2,2,2,49,6,1,150,1,1,3,0,0.0,2102,asalariado,1.096578e+09,9.753475e+08,121230867.0,256638442.0,2,112,0,2,1,1,0,0,1,0,1,2,1,0,1
2,2006,3,15,0,1,1,2,2,0,0,0,3,47,6,3,80,1,1,2,0,0.0,2257,autonomYRenta,8.331882e+08,6.341741e+08,199014135.0,386234221.0,2,82,0,0,2,1,0,0,2,0,0,1,1,0,1
3,2006,4,9,1,5,3,1,1,1,1,0,2,49,4,5,65,1,1,2,0,0.0,1661,asalariado,1.157571e+09,1.157571e+09,0.0,460834916.0,1,36,0,1,0,1,0,0,0,0,1,1,1,0,1
4,2006,5,9,0,1,1,1,1,2,2,0,4,38,5,1,120,1,1,3,0,0.0,1082,asalariado,1.935653e+09,1.532995e+09,402657609.0,835798459.0,1,72,1,1,1,1,0,0,1,1,0,2,1,0,0


In [45]:
# En NUMPERI, SUPERF Y NUMESTU, NUMOCU Y NUMACTI, reemplazar -9 por NaN
df_hogar_filtrado['NUMPERI'] = df_hogar_filtrado['NUMPERI'].replace(-9, None)
df_hogar_filtrado['SUPERF'] = df_hogar_filtrado['SUPERF'].replace(-9, None)
df_hogar_filtrado['NUMOCU'] = df_hogar_filtrado['NUMOCU'].replace(-9, None)
df_hogar_filtrado['NUMACTI'] = df_hogar_filtrado['NUMACTI'].replace(-9, None)
df_hogar_filtrado['COMITOT'] = df_hogar_filtrado['COMITOT'].replace(-9, None)

# Procesamiento del Fichero de Gastos

In [46]:
df_gastos_old

,ANOENC,NUMERO,CODIGO,GASTO,PORCEN,CANTIDAD,GASTOMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,FACTOR,Año
0,2011,1,1112,17975218,0,6410216.0,17975218.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
1,2011,1,1113,23693520,0,4661976.0,23693520.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
2,2011,1,1115,50528045,0,NaN,50528045.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
3,2011,1,1117,1942490,0,NaN,1942490.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
4,2011,1,1125,33205115,0,3884980.0,33205115.0,NaN,NaN,NaN,NaN,NaN,745065408,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10160616,2015,22130,12421,7459365,0,NaN,7459365.0,NaN,NaN,NaN,NaN,NaN,559172763,2015
10160617,2015,22130,12441,18323532,0,NaN,18323532.0,NaN,NaN,NaN,NaN,NaN,559172763,2015
10160618,2015,22130,12461,22559266,0,NaN,22559266.0,NaN,NaN,NaN,NaN,NaN,559172763,2015
10160619,2015,22130,12611,33608236,0,NaN,33608236.0,NaN,NaN,NaN,NaN,NaN,559172763,2015


In [47]:
# Unir los DataFrames de gastos
df_gastos = pd.concat([df_gastos_06_10, df_gastos_old, df_gastos], ignore_index=True)

df_gastos.head()

,ANOENC,NUMERO,CODIGO,GASTO,PORCEN,CANTIDAD,GASTOMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,FACTOR,Año,PORCENDES,PORCENIMP
0,2006,1,1112,29415522.0,0.0,10354264.0,29415522.0,NaN,NaN,NaN,NaN,NaN,902614262.0,2006,NaN,NaN
1,2006,1,1114,2494436.0,0.0,705973.0,2494436.0,NaN,NaN,NaN,NaN,NaN,902614262.0,2006,NaN,NaN
2,2006,1,1115,57113177.0,0.0,NaN,57113177.0,NaN,NaN,NaN,NaN,NaN,902614262.0,2006,NaN,NaN
3,2006,1,1117,6000766.0,0.0,NaN,6000766.0,NaN,NaN,NaN,NaN,NaN,902614262.0,2006,NaN,NaN
4,2006,1,1121,27085813.0,0.0,1882593.0,27085813.0,NaN,NaN,NaN,NaN,NaN,902614262.0,2006,NaN,NaN


In [48]:
df_gastos.tail(5)

,ANOENC,NUMERO,CODIGO,GASTO,PORCEN,CANTIDAD,GASTOMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,FACTOR,Año,PORCENDES,PORCENIMP
34655650,2023,20707,12521,106228.09,NaN,NaN,106228.09,NaN,NaN,NaN,NaN,NaN,544.759442,2023,0.0,0.0
34655651,2023,20707,12522,136189.86,NaN,NaN,136189.86,NaN,NaN,NaN,NaN,NaN,544.759442,2023,0.0,0.0
34655652,2023,20707,12541,163427.83,NaN,NaN,163427.83,NaN,NaN,NaN,NaN,NaN,544.759442,2023,0.0,0.0
34655653,2023,20707,12620,47712.61,NaN,NaN,47712.61,NaN,NaN,NaN,NaN,NaN,544.759442,2023,0.0,0.0
34655654,2023,20707,12702,834970.72,NaN,NaN,834970.72,NaN,NaN,NaN,NaN,NaN,544.759442,2023,0.0,0.0


In [49]:
# Lista de columnas sdeleccioandas del DataFrame según la estructura proporcionada
columnas_df_gastos = [
    "ANOENC", "NUMERO", "CODIGO", "GASTOMON", "FACTOR"
]

df_gastos_filtrado = df_gastos[columnas_df_gastos]
df_gastos_filtrado = df_gastos_filtrado.copy()

Procesamos el codigo

In [50]:
# Contar cuántas veces aparece el número 1 en la columna "NUMERO"
conteo_numero_1 = df_gastos['NUMERO'].eq(1).sum()

# Mostrar el resultado
print(f"El número 1 aparece {conteo_numero_1} veces en la columna 'NUMERO'.")


El número 1 aparece 1414 veces en la columna 'NUMERO'.


In [51]:
df_gastos_filtrado.head()

,ANOENC,NUMERO,CODIGO,GASTOMON,FACTOR
0,2006,1,1112,29415522.0,902614262.0
1,2006,1,1114,2494436.0,902614262.0
2,2006,1,1115,57113177.0,902614262.0
3,2006,1,1117,6000766.0,902614262.0
4,2006,1,1121,27085813.0,902614262.0


In [52]:
df_gastos_filtrado.tail()

,ANOENC,NUMERO,CODIGO,GASTOMON,FACTOR
34655650,2023,20707,12521,106228.09,544.759442
34655651,2023,20707,12522,136189.86,544.759442
34655652,2023,20707,12541,163427.83,544.759442
34655653,2023,20707,12620,47712.61,544.759442
34655654,2023,20707,12702,834970.72,544.759442


# Procesamiento de Indicadores Externos

In [53]:
import pandas as pd

In [54]:
tipo_interes_df = pd.read_csv("../../data/tipos_interes_bce.csv", sep="\t", encoding="utf-8")
temperaturas_medias_df = pd.read_csv("../../data/temperaturas_comunidades.csv", sep="\t", encoding="utf-8")
tasa_paro_df = pd.read_csv("../../data/tasa_paro_comunidad_2006_2023.csv", sep="\t", encoding="latin-1", decimal=",")
inflacion_df = pd.read_csv("../../data/inflacion_espana_2006_2023.csv", sep="\t", encoding="utf-8")
tasa_cambio_df = pd.read_csv("../../data/eur_usd_avg.csv", sep="\t", encoding="utf-8")
ipc_df = pd.read_csv("../../data/ipc_comunidad_año_media.csv", sep="\t", encoding="utf-8")


In [55]:
inflacion_df.head()

,Año,Comunidad Autónoma,Inflación (%)
0,2006,Andalucía,2.9
1,2006,Aragón,2.9
2,2006,Asturias,2.4
3,2006,Islas Baleares,2.8
4,2006,Canarias,2.0


In [56]:
tasa_cambio_df.head()

,Año,EUR_USD
0,2006,1.26
1,2007,1.37
2,2008,1.47
3,2009,1.39
4,2010,1.33


In [57]:
temperaturas_medias_df.head()

,CCAA,T_med
0,1,19.62
1,2,15.33
2,3,14.30
3,4,18.80
4,5,22.85


In [58]:
tasa_paro_df.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Periodo,Total
0,Total Nacional,01 Andalucía,2023*,18.2
1,Total Nacional,01 Andalucía,2022*,19.1
2,Total Nacional,01 Andalucía,2021*,21.9
3,Total Nacional,01 Andalucía,2020,22.3
4,Total Nacional,01 Andalucía,2019,21.2


In [59]:
ipc_df.head()

,Comunidades y Ciudades Autónomas,Año,Total_Medio
0,01 Andalucía,2006,81.547750
1,01 Andalucía,2007,83.804500
2,01 Andalucía,2008,87.164167
3,01 Andalucía,2009,86.760083
4,01 Andalucía,2010,88.313917


In [60]:
inflacion_df['Comunidad Autónoma'].value_counts()

Andalucía               18
Extremadura             18
Ceuta                   18
La Rioja                18
País Vasco              18
Navarra                 18
Murcia                  18
Madrid                  18
Galicia                 18
Comunidad Valenciana    18
Aragón                  18
Cataluña                18
Castilla-La Mancha      18
Castilla y León         18
Cantabria               18
Canarias                18
Islas Baleares          18
Asturias                18
Melilla                 18
Name: Comunidad Autónoma, dtype: int64

In [61]:
codigos_comunidad = {
    "Andalucía": 1, "Aragón": 2, "Asturias": 3, "Islas Baleares": 4,
    "Canarias": 5, "Cantabria": 6, "Castilla y León": 8, "Castilla-La Mancha": 7,
    "Cataluña": 9, "Comunidad Valenciana": 10, "Extremadura": 11, "Galicia": 12,
    "Madrid": 13, "Murcia": 14, "Navarra": 15,
    "País Vasco": 16, "La Rioja": 17, "Ceuta": 18, "Melilla": 19
}

# Agregar código de comunidad
inflacion_df["CCAA"] = inflacion_df["Comunidad Autónoma"].map(codigos_comunidad)



In [62]:
inflacion_df.head()


,Año,Comunidad Autónoma,Inflación (%),CCAA
0,2006,Andalucía,2.9,1
1,2006,Aragón,2.9,2
2,2006,Asturias,2.4,3
3,2006,Islas Baleares,2.8,4
4,2006,Canarias,2.0,5


In [63]:
tipo_interes_df.head()

,Año,Tipo_Interes
0,2006,3.5
1,2007,4.0
2,2008,2.5
3,2009,1.0
4,2010,1.0


In [64]:
ipc_df.head()

,Comunidades y Ciudades Autónomas,Año,Total_Medio
0,01 Andalucía,2006,81.547750
1,01 Andalucía,2007,83.804500
2,01 Andalucía,2008,87.164167
3,01 Andalucía,2009,86.760083
4,01 Andalucía,2010,88.313917


In [65]:
# Normalizar nombres de columna para facilitar el merge
tasa_paro_df = tasa_paro_df.rename(columns={"Comunidades y Ciudades Autónomas": "Comunidad_Autonoma", "Periodo": "Año", "Total": "Tasa_Paro"})
inflacion_df = inflacion_df.rename(columns={"Inflación (%)": "Inflacion"})
ipc_df = ipc_df.rename(columns={"Total_Medio": "IPC", "Comunidades y Ciudades Autónomas": "Comunidad_Autonoma"})

# Extraer el código de comunidad en tasa_paro_df
tasa_paro_df["CCAA"] = tasa_paro_df["Comunidad_Autonoma"].str.extract(r"^0?(\d{1,2})").astype(int)
ipc_df["CCAA"] = ipc_df["Comunidad_Autonoma"].str.extract(r"^0?(\d{1,2})").astype(int)

# En la columna "Año" de tasa_paro_df, eliminar si hay un asterisco al final
tasa_paro_df["Año"] = tasa_paro_df["Año"].str.replace(r"\*", "", regex=True)

# --- Unificar tipos a string --- 
tasa_paro_df["CCAA"]            = tasa_paro_df["CCAA"].astype(str)
temperaturas_medias_df["CCAA"]  = temperaturas_medias_df["CCAA"].astype(str)
inflacion_df["CCAA"]            = inflacion_df["CCAA"].astype(str)
ipc_df["CCAA"]                 = ipc_df["CCAA"].astype(str)

tasa_paro_df["Año"]             = tasa_paro_df["Año"].astype(str)
inflacion_df["Año"]             = inflacion_df["Año"].astype(str)
tipo_interes_df["Año"]          = tipo_interes_df["Año"].astype(str)
tasa_cambio_df["Año"]           = tasa_cambio_df["Año"].astype(str)
ipc_df["Año"]                  = ipc_df["Año"].astype(str)


# Unir los DataFrames por Código de Comunidad y Año
indicadores_externos = tasa_paro_df[["Año", "CCAA", "Tasa_Paro"]].merge(
    temperaturas_medias_df,
    on=["CCAA"],
    how="left"
).merge(
    inflacion_df[["CCAA", "Año", "Inflacion"]],
    on=["CCAA", "Año"],
    how="left"
).merge(
    tipo_interes_df,
    on="Año",
    how="left"
).merge(
    tasa_cambio_df,
    on="Año",
    how="left"
).merge(
    ipc_df[["CCAA", "Año", "IPC"]],
    on=["CCAA", "Año"],
    how="left"
)

In [66]:
indicadores_externos.rename(columns={'Año' : 'ANOENC'}, inplace=True)


In [67]:
indicadores_externos.head(5)

,ANOENC,CCAA,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC
0,2023,1,18.2,19.62,3.3,4.5,1.08,112.932083
1,2022,1,19.1,19.62,6.2,2.5,1.05,108.675167
2,2021,1,21.9,19.62,6.7,0.0,1.18,100.000000
3,2020,1,22.3,19.62,-0.3,0.0,1.14,96.956083
4,2019,1,21.2,19.62,0.5,0.0,1.12,97.157000


# Creamos nuestro datalake

In [68]:
import os
import pandas as pd

def crear_datalake(df_gastos, df_hogar, df_indicadores, output_dir="../../DataLake"):
    """
    Crea un DataLake estructurado por año, con tres archivos por año:
    - external_indicators.tsv: Todos los indicadores por comunidad autónoma.
    - homes.tsv: Todas las características del hogar excluyendo el año.
    - family_expenses.tsv: Toda la información de gastos excluyendo el año.

    Parámetros:
    - df_gastos (DataFrame): DataFrame con los datos de gasto (contiene Año y Número de hogar).
    - df_hogar (DataFrame): DataFrame con las características del hogar (contiene Año y CCAA).
    - df_indicadores (DataFrame): DataFrame con los datos de comunidad (contiene Año y CCAA).
    - output_dir (str): Directorio base donde se creará el datalake.
    """

    # Asegurar que la carpeta base del DataLake existe
    os.makedirs(output_dir, exist_ok=True)

    # Convertir tipos de datos para evitar problemas
    df_hogar['CCAA'] = df_hogar['CCAA'].astype(str)
    df_indicadores['CCAA'] = df_indicadores['CCAA'].astype(str)
    df_gastos['NUMERO'] = df_gastos['NUMERO'].astype(str)
    df_gastos['ANOENC'] = df_gastos['ANOENC'].astype(int)
    df_hogar['ANOENC'] = df_hogar['ANOENC'].astype(int)
    df_indicadores['ANOENC'] = df_indicadores['ANOENC'].astype(int)
    
    # Obtener todos los años únicos
    anos = df_hogar['ANOENC'].unique()

    for anoenc in anos:
        # Crear la carpeta del año si no existe
        carpeta_ano = os.path.join(output_dir, str(anoenc))
        os.makedirs(carpeta_ano, exist_ok=True)
        
        # Filtrar los datos por año
        df_indicadores_ano = df_indicadores[df_indicadores['ANOENC'] == anoenc].drop(columns=['ANOENC'])
        df_hogar_ano = df_hogar[df_hogar['ANOENC'] == anoenc].drop(columns=['ANOENC'])
        df_gastos_ano = df_gastos[df_gastos['ANOENC'] == anoenc].drop(columns=['ANOENC'])
        
        # Guardar los archivos en formato TSV con manejo de errores
        try:
            df_indicadores_ano.to_csv(os.path.join(carpeta_ano, "external_indicators.tsv"), sep="\t", index=False)
            df_hogar_ano.to_csv(os.path.join(carpeta_ano, "homes.tsv"), sep="\t", index=False)
            df_gastos_ano.to_csv(os.path.join(carpeta_ano, "family_expenses.tsv"), sep="\t", index=False)
        except OSError as e:
            print(f"❌ Error al guardar archivos en {carpeta_ano}: {e}")

    print(f"✅ DataLake creado en {output_dir}")


In [69]:
crear_datalake(df_gastos_filtrado, df_hogar_filtrado, indicadores_externos)

✅ DataLake creado en ../../DataLake
